In [7]:
import methods

In [7]:
from ctypes import *
import os
import numpy as np


class bench_result(Structure):
    _fields_ = [
        ("original_size", c_size_t),
        ("compressed_size", c_size_t),
        ("compression_time", c_double),
        ("decompression_time", c_double),
        ("max_error", c_double),
        ("mean_absolute_error", c_double),
    ]

    # https://stackoverflow.com/a/62011887/3901677
    def __repr__(self) -> str:
        values = ", ".join(f"{name}={value}" for name, value in self._asdict().items())
        return f"<{self.__class__.__name__}: {values}>"

    def _asdict(self) -> dict:
        return {field[0]: getattr(self, field[0]) for field in self._fields_}


lib = CDLL(os.path.abspath("../build/libcompression-benchmark.so"))
lib.reconstruct.argtypes = [POINTER(bench_result), c_char_p, c_char, c_void_p, c_int]
lib.reconstruct.restype = c_int


def reconstruct(method_name: str, array: np.ndarray):
    if(array.dtype == np.float32):
        dtype_char = ord('f')
    elif(array.dtype == np.float64):
        dtype_char = ord('d')
    else:
        raise Exception("Unsupported data type")
    buf = array.tobytes()
    results = bench_result()
    ret = lib.reconstruct(
        byref(results),
        create_string_buffer(method_name.encode("ascii")),
        dtype_char,
        buf,
        array.size,
    )
    if ret != 0:
        raise Exception("Reconstruct failed")
    return np.frombuffer(buf, dtype=array.dtype)


reconstruct("Sz3", np.array([1, 2, 3, 1, 5, 6, 7, 8, 9, 10], dtype="double"))


Using method Sz3
Compressing... done
Decompressing... done
Comparing... done
Original size:        80
Compressed size:      156
Ratio:                195%
Compression time:     0.000118327s
Decompression time:   6.3322e-05s
Mean Absolute Error:  0.44375
Max Error:            1


array([ 1.,  2.,  3.,  1.,  5.,  6.,  7.,  8.,  9., 10.])